# Prueba del API de Koalas

Vamos a probar el API de Koalas comparándolo con PySpark estándar

Necesitamos tener instalado Koalas en el entorno que ejecuta el Notebook, lo que puede hacerse simplemente con pip:

      pip install koalas

In [ ]:
# Ensure we are connected
spark

## Standard Spark

### Carga de datos

In [ ]:
# Read the data
censo = spark.read.format('parquet').load('data/censo-2011.parquet')

censo = censo.repartition(8)

In [ ]:
type(censo)

In [ ]:
%%time
censo.count()

In [ ]:
%%time
censo.limit(5).toPandas()

### Proceso

obtiene los emigrantes: aquellos cuya provincia de residencia no es la de nacimiento

In [ ]:
emigrantes = censo.filter( censo.CPRO != censo.CPRON )

In [ ]:
%%time
emigrantes.limit(5).toPandas()

In [ ]:
%%time
cem = emigrantes.count()

cem

In [ ]:
cem/censo.count()

In [ ]:
emprov = emigrantes.groupBy('CPRO').count().toPandas()

In [ ]:
emprov = emprov.rename(columns={'count':'emig'})
emprov

## Koalas
Repetimos las operaciones, esta vez usando Koalas

In [ ]:
import databricks.koalas as ks

# Ajusta una opción de configuración de Koalas
ks.set_option('compute.default_index_type', 'distributed')

### Carga de datos (Koalas)

Podemos leer directamente del fichero origen a un DataFrame de Koalas

In [ ]:
ks_censo1 = ks.read_parquet('data/censo-2011.parquet')

O, si tenemos ya un DataFrame de Spark, convertirlo a Koalas

In [ ]:
ks_censo2 = censo.to_koalas()

In [ ]:
type(ks_censo1)

### Proceso (Koalas)

In [ ]:
ks_emigrantes = ks_censo1[ks_censo1.CPRO != ks_censo1.CPRON]

In [ ]:
%%time

len(ks_emigrantes)

In [ ]:
%%time

ks_emigrantes.head(5)

In [ ]:
ks_emprov = ks_emigrantes.groupby('CPRO')[['CMUN']].count().rename(columns={'CMUN': 'NumEmig'}).reset_index()

In [ ]:
ks_emprov

### Proceso adicional

Añadimos nombres de provincia al resultado

In [ ]:
import pandas as pd
prov = ks.read_csv('data/codes-prov.csv', encoding='utf-8', dtype={'CPRO':str} )

In [ ]:
emprov2 = ks_emprov.merge(prov, on='CPRO').sort_values('NumEmig', ascending=False)

In [ ]:
emprov2